In [1]:
# install asteroid
!git clone https://github.com/asteroid-team/asteroid
%cd asteroid
!pip install -e .
!pip install asteroid

!pip install pytorch-lightning

Cloning into 'asteroid'...
remote: Enumerating objects: 7406, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 7406 (delta 78), reused 90 (delta 49), pack-reused 7242
Receiving objects: 100% (7406/7406), 5.87 MiB | 5.66 MiB/s, done.
Resolving deltas: 100% (4682/4682), done.
/content/asteroid
Obtaining file:///content/asteroid
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 72 kB 1.4 MB/s 
     |████████████████████████████████| 925 kB 32.6 MB/s 
     |████████████████████████████████| 596 kB 65.2 MB/s 
     |████████████████████████████████| 77 kB 9.8 MB/s 
     |████████████████████████████████| 58 kB 8.3 MB/s 
     |████████████████████████████████| 90 kB 13.7 MB/s 
     |████████████████████████████████| 136 kB 71.7 MB/s 
     |████████████████████████████████| 829 kB 59.4 MB/s 
     |█████████████

In [2]:
import torch
from torch import optim
from pytorch_lightning import Trainer
from asteroid.models import ConvTasNet
from asteroid.losses import pairwise_neg_sisdr, PITLossWrapper
from asteroid.data import LibriMix
from asteroid.engine import System
from torch.utils.data import DataLoader

# to enable cuda, you have to do a manual toggle in the settings
# menu 'runtime->change runtime->hardware accelerator->gpu'
torch.cuda.is_available()

True

In [3]:
# import custom data set from py in google drive
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.insert(0,'/content/drive/My Drive')
from custom_dataset_sandbox import CustomMixture 

Mounted at /content/drive


In [4]:
csv_path = '/content/drive/MyDrive/output2/'
%cd /content/drive/MyDrive/output2/
train_loader = DataLoader(CustomMixture(csv_path, "train_metadata.csv"), num_workers=8, batch_size=2)
val_loader = DataLoader(CustomMixture(csv_path, "val_metadata.csv"), num_workers=8, batch_size=2)

/content/drive/MyDrive/output2


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [5]:
# re-instantiate the class objects
model = ConvTasNet(n_src=2)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss = PITLossWrapper(pairwise_neg_sisdr, pit_from="pw_mtx") # PITLossWrapper works with any loss function.

# access the .ckpt file from the drive
checkpoint = torch.load('/content/drive/MyDrive/output2/lightning_logs/version_6/checkpoints/epoch=39-step=89799.ckpt')

In [6]:
''' load the stored data into the class objects '''
print(checkpoint.keys())

# adjust 'state_dict' to fit what model requires
model_state_dict = {}
for key in checkpoint['state_dict'].keys():
  model_state_dict[key[6:]] = checkpoint['state_dict'].get(key)
# print(checkpoint['state_dict'].keys())
# print(model_state_dict.keys())
model.load_state_dict(model_state_dict)

# adjust 'optimizer_states' to fit what optimizer requires
# optimizer is expecting a dict, but 'optimizer_states' is a list...
# the list is of len() 1, passing [0] seems to fix everything
optimizer.load_state_dict(checkpoint['optimizer_states'][0])
# print(len(checkpoint['optimizer_states']))

epoch = checkpoint['epoch']

# I am electing to leave this line commented out because:
# 1. there is no dict key 'loss'
# 2. unlike model and optimizer, this class object is not being edited,
#    but overwritten
# loss = checkpoint['loss']

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'callbacks', 'optimizer_states', 'lr_schedulers', 'training_config'])


In [7]:
# # eval mode
# model.eval()

# import librosa
# audiofile, sr_rate = librosa.load('/content/drive/MyDrive/output/mixture/mixtureclip13268.wav') 
# prediction = model(torch.from_numpy(audiofile))

# import soundfile as sf
# sf.write('/content/drive/MyDrive/output/prediction_wavs/i2p2e39_3v_13268_wav1.wav', prediction.detach().numpy()[0], sr_rate)
# sf.write('/content/drive/MyDrive/output/prediction_wavs/i2p2e39_3v_13268_wav2.wav', prediction.detach().numpy()[1], sr_rate)

# audiofile, sr_rate = librosa.load('/content/drive/MyDrive/output/mixture/mixtureclip4053.wav') 
# prediction = model(torch.from_numpy(audiofile))

# sf.write('/content/drive/MyDrive/output/prediction_wavs/i2p2e39_2v_4053_wav1.wav', prediction.detach().numpy()[0], sr_rate)
# sf.write('/content/drive/MyDrive/output/prediction_wavs/i2p2e39_2v_4053_wav2.wav', prediction.detach().numpy()[1], sr_rate)

# audiofile, sr_rate = librosa.load('/content/drive/MyDrive/output/mixture/mixtureclip6587.wav') 
# prediction = model(torch.from_numpy(audiofile))

# sf.write('/content/drive/MyDrive/output/prediction_wavs/i2p2e39_1v_6587_wav1.wav', prediction.detach().numpy()[0], sr_rate)
# sf.write('/content/drive/MyDrive/output/prediction_wavs/i2p2e39_1v_6587_wav2.wav', prediction.detach().numpy()[1], sr_rate)


In [8]:
system = System(model, optimizer, loss, train_loader, val_loader)

In [ ]:
trainer = Trainer(max_epochs=40, gpus=1, auto_select_gpus=True, log_every_n_steps=1)
trainer.fit(system)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type           | Params
---------------------------------------------
0 | model     | ConvTasNet     | 5.1 M 
1 | loss_func | PITLossWrapper | 0     
---------------------------------------------
5.1 M     Trainable params
0         Non-trainable params
5.1 M     Total params
20.202    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
model_path = '/content/drive/MyDrive/output2/model_saves/iteration1_part3.pt'
torch.save(model.state_dict(), model_path)